## Проверяем насколько точно моделька gpt-4o-mini может вычислить предвзятость в текстах

In [1]:
import pandas as pd
import openai
from sklearn.metrics import accuracy_score, classification_report
from config import api_key 

### Мы собрали данные (предложения) из двух датасетов: [BABE](https://www.kaggle.com/datasets/timospinde/babe-media-bias-annotations-by-experts/data) и [MBIC](https://www.kaggle.com/datasets/timospinde/mbic-a-media-bias-annotation-dataset?resource=download). Они оба содержат предложения из медиа, которые отнесены к одному из двух классов: Biased (1) и Non-biased (0). Мы собрали по 150 предложений с каждого датасета и получился один датасет из 300 предложений с метками 0 и 1.

In [10]:
openai.api_key = api_key

# Загрузка данных
df = pd.read_csv("/Users/Anna/Documents/python/ML_PROJECT/data_test_8.csv", encoding='utf-8', sep=';')  
df.head()

,sentence,label_bias
0,YouTube is making clear there will be no “birt...,1
1,The increasingly bitter dispute between Americ...,0
2,So while there may be a humanitarian crisis dr...,1
3,A professor who teaches climate change classes...,0
4,"Looking around the United States, there is nev...",1


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentence    300 non-null    object
 1   label_bias  300 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.8+ KB


### Разделим метки и предложения для дальнейшей оценки модели

In [13]:
texts = df["sentence"].tolist()
labels = df["label_bias"].tolist()

### Посмотрим, как модель вычисляет предвзятость с разной температурой

Temperature = 0

In [15]:
# функция для классификации через GPT
def classify_text(text):
    prompt = f"""Classify the following text into one of two classes.: Biased (1) and Non-biased (0).
Answer with only one number (0 or 1), with no explanation.

Text: "{text}"
"""
    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
)
    reply = response.choices[0].message.content.strip()
    return int(reply)


predicted_labels = [classify_text(text) for text in texts]


In [17]:
# Отчет
accuracy = accuracy_score(labels, predicted_labels)
print("Точность (accuracy):", accuracy)
print("\nОтчет по метрикам:")
print(classification_report(labels, predicted_labels))

Точность (accuracy): 0.7766666666666666

Отчет по метрикам:
              precision    recall  f1-score   support

           0       0.87      0.68      0.76       160
           1       0.71      0.89      0.79       140

    accuracy                           0.78       300
   macro avg       0.79      0.78      0.78       300
weighted avg       0.80      0.78      0.78       300



Temperature = 0.5

In [18]:
def classify_text(text):
    prompt = f"""Classify the following text into one of two classes.: Biased (1) and Non-biased (0).
Answer with only one number (0 or 1), with no explanation.

Text: "{text}"
"""
    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
)
    reply = response.choices[0].message.content.strip()
    return int(reply)


predicted_labels = [classify_text(text) for text in texts]

accuracy = accuracy_score(labels, predicted_labels)
print("Точность (accuracy):", accuracy)
print("\nОтчет по метрикам:")
print(classification_report(labels, predicted_labels))

Точность (accuracy): 0.77

Отчет по метрикам:
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       160
           1       0.70      0.88      0.78       140

    accuracy                           0.77       300
   macro avg       0.78      0.78      0.77       300
weighted avg       0.79      0.77      0.77       300



Temperature = 1

In [19]:
def classify_text(text):
    prompt = f"""Classify the following text into one of two classes.: Biased (1) and Non-biased (0).
Answer with only one number (0 or 1), with no explanation.

Text: "{text}"
"""
    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
)
    reply = response.choices[0].message.content.strip()
    return int(reply)


predicted_labels = [classify_text(text) for text in texts]

accuracy = accuracy_score(labels, predicted_labels)
print("Точность (accuracy):", accuracy)
print("\nОтчет по метрикам:")
print(classification_report(labels, predicted_labels))

Точность (accuracy): 0.77

Отчет по метрикам:
              precision    recall  f1-score   support

           0       0.87      0.67      0.76       160
           1       0.70      0.89      0.78       140

    accuracy                           0.77       300
   macro avg       0.79      0.78      0.77       300
weighted avg       0.79      0.77      0.77       300



### Выводы: температура не сильно влияет на качество оценки модели, почти везде это 0,77